<a href="https://colab.research.google.com/github/deborahjori/Python/blob/main/Analytics_Northwuind.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importando bibliotecas

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

#Importando fontes de dados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
categories             =  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deborah/Desafio_Indicium/Fontes de Dados/categories.csv', sep=';', usecols = ['category_id','category_name','description'])
##customer_customer_demo =  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deborah/Desafio_Indicium/Fontes de Dados/customer_customer_demo.csv', sep=';')
##customer_demographics  =  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deborah/Desafio_Indicium/Fontes de Dados/customer_demographics.csv', sep=';')
customers              =  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deborah/Desafio_Indicium/Fontes de Dados/customers.csv', sep=';', usecols = ['customer_id','company_name','city','country'])
employee_territories   =  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deborah/Desafio_Indicium/Fontes de Dados/employee_territories.csv', sep=';')
employees              =  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deborah/Desafio_Indicium/Fontes de Dados/employees.csv',sep=';', usecols = ['employee_id','last_name','first_name','hire_date','title','city','country','region'] )
order_details          =  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deborah/Desafio_Indicium/Fontes de Dados/order_details.csv', sep=';' )
orders                 =  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deborah/Desafio_Indicium/Fontes de Dados/orders.csv' , sep=';', usecols = ['order_id','customer_id', 'employee_id','order_date','required_date','shipped_date','ship_via','freight', 'ship_city','ship_country'])
products               =  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deborah/Desafio_Indicium/Fontes de Dados/products.csv', sep=';')
region                 =  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deborah/Desafio_Indicium/Fontes de Dados/region.csv', sep=';')
shippers               =  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deborah/Desafio_Indicium/Fontes de Dados/shippers.csv', sep=';', usecols =['company_name'])
suppliers              =  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deborah/Desafio_Indicium/Fontes de Dados/suppliers.csv', sep=';', usecols = ['supplier_id','company_name','city','country'])
territories            =  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deborah/Desafio_Indicium/Fontes de Dados/territories.csv', sep=';')
us_states              =  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deborah/Desafio_Indicium/Fontes de Dados/us_states.csv',sep=';', usecols =['state_name','state_abbr','state_region'])

Calculando faturamento

In [ ]:
##Criando coluna de faturamento
order_details['sales'] = (order_details['unit_price']*order_details['quantity']) - ((order_details['unit_price']*order_details['quantity'])*order_details['discount'])

#Criando dimensões

##Dimensão Produto

Considerando units_in_stock e units_on_order, como estoque atual

In [ ]:
products['total_stock'] = products['units_in_stock'] + products['units_on_order']

Enriquecendo com dados de categorias

In [ ]:
products_enriched = pd.merge(products, categories, how = 'left', on = 'category_id')

Enriquecendo com dados de fornecedores

In [ ]:
products_enriched = pd.merge(products_enriched, suppliers, how = 'left', on = 'supplier_id')

Calculando dados de faturamento por produto

In [ ]:
products_summarized = order_details.groupby("product_id")["sales"].sum()

Enriquecendo com dados de faturamento por produto

In [ ]:
products_enriched = pd.merge(products_enriched, products_summarized, how = 'left', on = 'product_id')

Normalizando dimensão produto

In [ ]:
products_enriched.drop(columns=['category_id','supplier_id',], inplace = True)

In [ ]:
products_enriched.rename(columns={ 'company_name' : 'supplier',
                                   'city'         : 'supplier_city',
                                   'country'      : 'supplier_country',
                                   'description'  : 'category_description',
                                   'sales'        : 'product_sales'
                                }, inplace = True)

In [ ]:
for t in products_enriched['discontinued']:
  if t == 0:
    text = 'Não'
  else:
    text = 'Sim'

  products_enriched['discontinued'].replace(t, text, inplace = True)

Formatando dados financeiros

In [ ]:
products_enriched['product_sales'] = products_enriched['product_sales'].round(2)           

In [ ]:
products_enriched['product_sales'] = [str(format(value, '.2f').replace(".", ",")) for value in products_enriched['product_sales']]

In [ ]:
products_enriched['unit_price'] = [str(format(value, '.2f').replace(".", ",")) for value in products_enriched['unit_price']]

##Dimensão Funcionários por Territórios

Enriquecendo com dados de territórios 

In [ ]:
employee_territory_enriched = pd.merge(employee_territories, territories, how = 'left', on = 'territory_id')

Enriquecendo com dados de regiões

In [ ]:
employee_territory_enriched = pd.merge(employee_territory_enriched, region, how = 'left', on = 'region_id')

In [ ]:
employee_territory_enriched.drop(columns=['territory_id','region_id'], inplace = True)

##Dimensão Funcionários

In [ ]:
employee_enriched = employees

Unificando nomes

In [ ]:
employee_enriched['employee_name'] = employee_enriched['first_name'] + ' ' + employee_enriched['last_name'] 

Normalizando dimensão funcionários

In [ ]:
employee_enriched.drop(columns=['last_name','first_name'], inplace = True)

##Dimensão Clientes

In [ ]:
orders_summarized = orders.groupby("customer_id")["order_id"].count()


In [ ]:
customers_enriched = pd.merge(customers, orders_summarized, how = 'left', on = 'customer_id')

In [ ]:
customers_enriched.rename(columns={ 'order_id' : 'qtd_order'                                 
                                }, inplace = True)

##Dimensão Vendas

Sumarizando faturamento por ordem de venda

In [ ]:
order_details_summarized = order_details.groupby("order_id")["sales"].sum()

Enriquecendo com dados de faturamento sumarizados

In [ ]:
orders_enriched = pd.merge(orders, order_details_summarized, how = 'left', on = 'order_id')

Enriquecendo com dados de clientes

In [ ]:
orders_enriched = pd.merge(orders_enriched, customers, how = 'left', on = 'customer_id')

Normalizando dimensão Vendas

In [ ]:
orders_enriched.rename(columns={ 'company_name' : 'customer',
                                 'city'         : 'customer_city',
                                 'country'      : 'customer_country',
                                 'sales'      : 'order_sales',
                                }, inplace = True)

Formatando dados financeiros

In [ ]:
orders_enriched['freight'] = [str(format(value, '.2f').replace(".", ",")) for value in orders_enriched['freight']]

In [ ]:
orders_enriched['order_sales'] = orders_enriched['order_sales'].round(2)                 

In [ ]:
orders_enriched['order_sales'] = [str(format(value, '.2f').replace(".", ",")) for value in orders_enriched['order_sales']]

#Download das dimensões

In [ ]:
products_enriched.to_csv('/content/product_dimension.csv', sep = ';', index = False)

In [ ]:
employee_enriched.to_csv('/content/employee_dimension.csv', sep = ';', index = False)

In [ ]:
employee_territory_enriched.to_csv('/content/employee_territory_dimension.csv', sep = ';', index = False)

In [ ]:
customers.to_csv('/content/customers_dimension.csv', sep = ';', index = False)

In [ ]:
orders_enriched.to_csv('/content/order_dimension.csv', sep = ';', index = False)